# 데이터 변환

* Last updated 20191128THR1101 20181009_20170421_20161125

## S.1 학습내용

### S.1.1 목표

* Spark ETL을 할 수 있다.
* Spark를 사용하여 구조적 데이터를 분석할 수 있다.
* Spark를 사용하여 텍스트 분석을 할 수 있다.


### S.1.2 목차

* S.2 IPython Notebook에서 SparkSession 생성하기
* S.3 데이터 타잎과 변환
* S.3.1 vectors
* S.3.2 sparse vectors
* S.3.3 labeled point
* S.3.4 libsvm format
* S.4 통계

* S.5 DataFrame 변환 
* S.5.1 텍스트 변환
* S.5.2 Python을 사용한 단어 빈도 계산
* S.5.3 Spark

* S.5.4 StringIndexer
* S.5.5 Tokenizer
* S.5.6 RegTokenizer
* S.5.7 Stopwords
* S.5.8 CountVectorizer
* S.5.9 TF-IDF
* S.5.10 Word2Vec
* S.5.11 NGram
* S.5.12 연속데이터의 변환
* 5.5.13 VectorAssembler
* S.5.14 Pipeline

### S.1.3 문제 

* 문제 S-1: 훈련데이터 만들기
* 문제 S-2: Kolmogorov-Smirnov 검증
* 문제 S-3: 평균, 표준편차와 같은 기본 통계 값을 구한다.
* 문제 S-4: 연설문을 기계학습하기 위해 변환

## S.2 IPython Notebook에서 SparkSession 생성하기


In [2]:
import os
import sys 
os.environ["SPARK_HOME"]=os.path.join(os.environ['HOME'],'Downloads','spark-2.0.0-bin-hadoop2.7')
os.environ["PYLIB"]=os.path.join(os.environ["SPARK_HOME"],'python','lib')
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'py4j-0.10.1-src.zip'))
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'pyspark.zip'))

In [1]:
import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

In [2]:
print spark.version

2.2.0


## S.3 데이터 타잎과 변환

Spark는 예측, 분류, 군집화, 추천 등 여러 기계학습 모델을 제공하고, 이를 위해 데이터를 추출, 변환하는 이른바 **ETL** (Extract, Transform, Load)이 필수적이다. 기계학습에는 무작정 데이터를 우겨 넣어서는 작동되지 않는다. 입력될 데이터를 일정한 형식을 가지도록 구성하는 것이 필요하다. 여기에 필요한 데이터 타잎 **```Vector```**, **```Labeled Point```**, **```Matrix```**를 설명한다. 특히 RDD는 label과 features를 가지고 있는 Labeled Point로 구성해야 한다.

구분 | 설명
----------|----------
```Vector``` | ```numpy vector```와 같은 기능을 한다. **dense**와 **sparse** vector로 구분한다.
```Labeled Point``` | 분류를 의미하는 클래스 또는 **label**과 속성 **features** 이 묶인 구조로서, supervised learning에 사용된다.
```Matrix``` | ```numpy matrix```와 같은 특징을 가진다.

이러한 데이터 타잎은 Spark에서는 **```RDD mllib```** , **```DataFrame ml```** 패키지 별로 제공되므로, 식별하여 사용한다.
```ml``` 패키지를 사용할 경우에는 자신의 ```pyspark.ml.linalg.Vector``` 등을 사용해야 한다. ```mllib```도 마찬가지이다.

패키지 | 설명 | 데이터 타잎 예
-------|-------|-----
```mllib``` | RDD API | ```pyspark.mllib.linalg.Vector```, ```pyspark.mllib.linalg.Matrix```
```ml``` | DataFrame API | ```pyspark.ml.linalg.Vector```, ```pyspark.ml.linalg.Matrix```


### S.3.1 vectors

행렬 **Vector**는 **dense**와 **sparse**로 구분할 수 있다.
numpy array를 사용해도 dense vector를 만들 수 있다. Spark 내부적으로 **numpy.array**를 사용하고 있다.

In [1]:
import numpy as np

dv = np.array([1.0, 2.1, 3])

Spark에서는 **```RDD mllib```** , **```DataFrame ml```**의 **Vectors**를 사용하여 dense vectors를 만들 수 있다.

In [2]:
from pyspark.mllib.linalg import Vectors

dv1mllib=Vectors.dense([1.0, 2.1, 3])
print dv1mllib, type(dv1mllib)

[1.0,2.1,3.0] <class 'pyspark.mllib.linalg.DenseVector'>


In [2]:
from pyspark.ml.linalg import Vectors

dv1ml=Vectors.dense([1.0, 2.1, 3])
print dv1ml

[1.0,2.1,3.0]


dense vectors는 numpy array와 같은 특징을 가진다.
인덱스로 값을 읽을 수 있다. 또한 반복문에서 사용할 수 있다.

In [4]:
for e in dv1ml:
    print e,

1.0 2.1 3.0


보통 벡터와 같이 **product**, **dot**, **norm**과 같은 벡터 연산을 할 수도 있다.
결과 값은 numpy와 동일하다.

In [5]:
print dv1ml.dot(dv1ml)

14.41


In [6]:
np.dot(dv,dv)

14.41

더하기, 빼기, 곱하기, 나누기 연산은 항목별로 실행한다.

In [7]:
print dv1ml*dv1ml

[1.0,4.41,9.0]


### S.3.2 Sparse vectors

행렬에는 0 값이 많이 존재하기 때문에, 0값이 아닌 **NZ Nonzero**를 제거하여 저장하면 훨씬 효율적이다.
**sparse**는 실제 **값이 없는 요소, '0'을 제거**하여 만든 vector이다.
Spark에서 type field (1 바이트 길이)를 통해 식별한다 (0: sparse, 1: dense)

#### 1차원 Sparse Vectors

예를 들어, 다음은 1차원 dense vector이다.
```python
[160,69,24]
```

sparse vector로 표현하면, 값이 없는 요소 Nonzero가 없으니 더 복잡해 보인다.
3은 컬럼 갯수, 0,1,2는 값이 있는 컬럼, [160.0,69.0,24.0]는 실제 값을 의미한다.
```python
(3,[0,1,2],[160.0,69.0,24.0])
```

dense vector | sparse vector
----------|----------
**모든** 행열 값을 가지고 있다. | **인덱스 및 값**의 배열을 별도로 가진다.
빈 값이 별로 없는 경우. | 빈 값 Nonzero가 많은 경우 사용. 

sparse vectors는 값 중에 0이 포함된 경우 이를 생략한다.
```toArray()``` 함수를 사용하면 sparse에서 dense로 벡터를 변환할 수 있다.


In [3]:
sv1 = Vectors.sparse(3, [1, 2], [1.0, 3.0])
print sv1.toArray()  #convert sparse to array

[ 0.  1.  3.]


#### Sparse Vectors의 배열 방식 표현

```python
[1 0 2]
[0 0 3]
[4 5 6]

행 | 0 | 0 | 1 | 2 | 2 | 2
열 | 0 | 2 | 2 | 0 | 1 | 2
값 | 1 | 2 | 3 | 4 | 5 | 6
```

행을 보면 0번째에 '1','2' 1번째에 '3', 2번째에 '4','5','6'이므로 **0,0,1,2,2,2**
열을 보면 0번째에 '1', 2번째 '2','3', 0번째 '4', 1번째 '5', 2번째 '6'이므로 **0,2,2,0,1,2**

**행, 열, 데이터를 한 쌍**으로 읽으면 된다.
즉 행 0, 열 0의 위치에 1, 행 0, 열 2의 위치에 2. 이런 식으로 6개의 데이터가 있다.


In [22]:
import numpy as np
import scipy.sparse as sps

row = np.array([0, 0, 1, 2, 2, 2])
col = np.array([0, 2, 2, 0, 1, 2])
data = np.array([1, 2, 3, 4, 5, 6])
mtx = sps.csc_matrix((data, (row, col)), shape=(3, 3))
print mtx.todense()   


[[1 0 2]
 [0 0 3]
 [4 5 6]]


#### Sparse Vectors의 CSR (Compressed Sparse Row)

**```Compressed Sparse Column```** 또는 Yale Format이라고 한다. 다음 5개의 값으로 표현된다.
* int 행 개수
* int 열 갯수
* int[] colPtrs IA:
    * IA[0] = 0 처음은 0으로 놓고
    * IA[i] = IA[i-1] + (i-1)번째 행의 NNZ (Num of Nonzeroes)
    * 마지막은 NNZ (No of NonZeros)가 된다.
* int[] rowIndices JA: 각 요소의 행(컬럼) 인덱스
* double values

```python
[ 1.,  0.,  0.,  0.]
[ 2.,  3.,  0.,  0.]
[ 0.,  0.,  5.,  0.]
[ 0.,  4.,  6.,  0.]
[ 0.,  0.,  7.,  0.]
[ 0.,  0.,  0.,  8.]
```

* 6은 행 갯수
* 4능 열 갯수
* [0, 2, 4, 7, 8]은 **0**:IA[0]=0으로 시작, **2**:IA[1]=IA[0]+2, **4**:IA[2]=IA[1]+2 **7**:IA[3]=IA[2]+3, **8**:IA[4]=IA[3]+1
* 인덱스 [0, 1, 1, 3, 2, 3, 4, 5]은 컬럼순, 즉 위아래로 읽으면 1:**0**행, 2:**1**행, 3:**1**행, 4:**3**행, 5:**2**행, 6:**3**행, 7:**4**행, 8:**5**행
* 실제 값 [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]

dense Matrics는 행의 갯수, 열의 갯수, 실제 값을 넣어주면 생성된다.

In [13]:
dm = Matrices.dense(6,4,[1, 2, 0, 0, 0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 5, 6, 7, 0, 0, 0, 0, 0, 0, 8])
dm.toArray()

array([[ 1.,  0.,  0.,  0.],
       [ 2.,  3.,  0.,  0.],
       [ 0.,  0.,  5.,  0.],
       [ 0.,  4.,  6.,  0.],
       [ 0.,  0.,  7.,  0.],
       [ 0.,  0.,  0.,  8.]])

In [14]:
dm.toSparse()

SparseMatrix(6, 4, [0, 2, 4, 7, 8], [0, 1, 1, 3, 2, 3, 4, 5], [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0], False)

In [5]:
from pyspark.mllib.linalg import Matrix, Matrices
# Create a dense matrix ((1.0, 2.0), (3.0, 4.0), (5.0, 6.0))
[ 1.,  4.], [ 2., 5.], [ 3.,  6.]

([1.0, 4.0], [2.0, 5.0], [3.0, 6.0])

* 아래는 행3, 열2개 행열로 [9., 0.], [0., 8.], [0., 6.] 값이다.
* 0,1,3: **0**:IA[0]=0으로 시작, **1**:IA[1]=IA[0]+1 (요소 9), **3**:IA[2]=IA[1]+2 (요소 6,8)
* 0,2,1: 9:**0**행, 6:**2**행, 8:**1**행

In [7]:
sm = Matrices.sparse(3, 2, [0, 1, 3], [0, 2, 1], [9, 6, 8])

In [8]:
d=sm.toDense()
print d

DenseMatrix([[ 9.,  0.],
             [ 0.,  8.],
             [ 0.,  6.]])


#### RowMatrix

local matrix는 ```pyspark.mllib.linalg.Matrix, Matrices```를 사용한다.
distributed matrix는 제공되는 패키지에서 보듯이 분산해서 사용할 수 있다.
리스트를 묶어 여러 행으로 구성된 벡터를 생성한다.

In [3]:
p = [[1.0,2.0,3.0],[1.1,2.1,3.1],[1.2,2.2,3.3]]
my=spark.sparkContext.parallelize(p)

In [4]:
my.collect()

[[1.0, 2.0, 3.0], [1.1, 2.1, 3.1], [1.2, 2.2, 3.3]]

RowMatrix는 ```pyspark.mllib.linalg.distributed```에서 제공되는 분산벡터이다.

In [5]:
from pyspark.mllib.linalg.distributed import RowMatrix
rm=RowMatrix(my)

In [7]:
print type(rm)

<class 'pyspark.mllib.linalg.distributed.RowMatrix'>


In [6]:
rm.rows.collect()

[DenseVector([1.0, 2.0, 3.0]),
 DenseVector([1.1, 2.1, 3.1]),
 DenseVector([1.2, 2.2, 3.3])]

### S.3.3 Labeled Point

#### label, features로 구성

**분류** 및 **회귀분석**에 사용되는 데이터 타잎이다.
**'label'**과 **'features'**로 구성된다.

구분 | 설명
-----|-----
label | supervised learning에서 '구분 값'으로 사용한다. 데이터타잎은 'Double'
features | **sparse**, **dense** 모두 사용할 수 있다.

label 1.0, features [1.0, 2.0, 3.0]으로 LabeledPoint를 만들어 보자.

구분 | 예제 | 설명
-----|-----|-----
label | 1.0 | 구분 값으로 Double 데이터 타잎
features | [1.0, 2.0, 3.0] | **dense vector**

In [29]:
from pyspark.mllib.regression import LabeledPoint

print LabeledPoint(1.0, [1.0, 2.0, 3.0])

(1.0,[1.0,2.0,3.0])


In [8]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors

print LabeledPoint(1992, Vectors.sparse(10, {0: 3.0, 1:5.5, 2: 10.0}))

(1992.0,(10,[0,1,2],[3.0,5.5,10.0]))


서로 다른 패키지의 데이터타잎 **```mllib LabeledPoint```**와 **```ml Vectors```**를 혼용하면, 형변환 오류가 발생한다.
이러한 오류는 패키지를 혼용하지 않으면 된다.

```python
Cannot convert type <class 'pyspark.ml.linalg.DenseVector'> into Vector
```

**```dv1mllib```**은 앞서 **```mllib```**로부터 생성된 dense vector이다.

In [31]:
from pyspark.mllib.regression import LabeledPoint

LabeledPoint(1.0, dv1mllib)

LabeledPoint(1.0, [1.0,2.1,3.0])

**```dv1ml```**은 앞서 **```ml```**로부터 생성된 dense vector이다.
```mllib```에서 사용하려면, **```Vectors.fromML()```**를 사용해 ```ml```의 Vectors를 읽어서 ```mllib```로 변환하여 혼용을 막는다.

In [32]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors

LabeledPoint(1.0, Vectors.fromML(dv1ml))

LabeledPoint(1.0, [1.0,2.1,3.0])

#### DataFrame에서 Labeled Point

* Python list에서 DataFrame 생성

In [3]:
p = [[1,[1.0,2.0,3.0]],[1,[1.1,2.1,3.1]],[0,[1.2,2.2,3.3]]]
trainDf=spark.createDataFrame(p)
trainDf.collect()

[Row(_1=1, _2=[1.0, 2.0, 3.0]),
 Row(_1=1, _2=[1.1, 2.1, 3.1]),
 Row(_1=0, _2=[1.2, 2.2, 3.3])]

* Python list를 LabeledPoint로 생성하면, 'label'과 'features'의 명칭을 가지도록 생성된다.

In [16]:
from pyspark.mllib.regression import LabeledPoint
p = [LabeledPoint(1,[1.0,2.0,3.0]),
     LabeledPoint(1,[1.1,2.1,3.1]),
     LabeledPoint(0,[1.2,2.2,3.3])]
trainDf=spark.createDataFrame(p)
trainDf.collect()

[Row(features=DenseVector([1.0, 2.0, 3.0]), label=1.0),
 Row(features=DenseVector([1.1, 2.1, 3.1]), label=1.0),
 Row(features=DenseVector([1.2, 2.2, 3.3]), label=0.0)]

mllib.linalg.Vectors를 사용하여 DataFrame을 생성해 보자.

In [10]:
from pyspark.mllib.linalg import Vectors

trainDf = spark.createDataFrame([
    (1.0, Vectors.dense([0.0, 1.1, 0.1])),
    (0.0, Vectors.dense([2.0, 1.0, 1.0])),
    (0.0, Vectors.dense([2.0, 1.3, 1.0])),
    (1.0, Vectors.dense([0.0, 1.2, 0.5]))], ["label", "features"])
trainDf.collect()

[Row(label=1.0, features=DenseVector([0.0, 1.1, 0.1])),
 Row(label=0.0, features=DenseVector([2.0, 1.0, 1.0])),
 Row(label=0.0, features=DenseVector([2.0, 1.3, 1.0])),
 Row(label=1.0, features=DenseVector([0.0, 1.2, 0.5]))]

schema를 사용해서 DataFrame을 생성해 보자.
* 'label'은 **DoubleType**
* 'features'는 **VectorType**

In [17]:
from pyspark.mllib.linalg import SparseVector, VectorUDT
from pyspark.sql.types import StructType, StructField, DoubleType
_rdd = spark.sparkContext.parallelize([
    (0.0, SparseVector(4, {1: 1.0, 3: 5.5})),
    (1.0, SparseVector(4, {0: -1.0, 2: 0.5}))])

schema = StructType([
    StructField("label", DoubleType(), True),
    StructField("features", VectorUDT(), True)
])

In [18]:
trainDf=_rdd.toDF(schema)
trainDf.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



#### sparse에서 dense vector로 변환

방금 생성한 ```trainDf```는 sparse vector이다.
사용자 함수udf User Defined Type를 사용하여 sparse vector를 dense vector로 변환해 보자.
바로 변환할 수 있는 함수 **toDense() 함수를 지원하지 않으므로**, **sparse vector를 ```toArray()``` 함수를 사용해서 dense vector로 변환**한다.

또 ```trainDf```는 RDD의 mllib에서 변환된 데이터이므로, mllib 라이브러리를 사용한다.

구분 | 설명
-----|-----
from pyspark.mllib.linalg import DenseVector,VectorUDT | mllib
from pyspark.ml.linalg import DenseVector,VectorUDT | ml

다음 명령문을 요소별로 하나씩 설명한다.
```python
udf(lambda x: DenseVector(x.toArray()), VectorUDT())
```
명령문 | 설명
-----|-----
```udf()``` | 사용자정의 함수
```x.toArray()``` | sparse vector로 구성된 trainDf.features를 **```toArray()```**를 사용하여 array로 변환한다.
```VectorUDT()``` | 타잎을 지정한다. 지정하지 않으면 ```StringType```을 기본으로, 자동으로 변환된다.
```DenseVector()``` | array를 dense vector로 변환한다.

In [22]:
from pyspark.sql.functions import udf
#from pyspark.ml.linalg import DenseVector, VectorUDT
from pyspark.mllib.linalg import DenseVector, VectorUDT
#myudf=udf(lambda x: Vectors.dense(x), VectorUDT())
#myudf=udf(lambda x: Vectors.dense(x))
myudf=udf(lambda x: DenseVector(x.toArray()), VectorUDT())
_trainDf2=trainDf.withColumn('dvf',myudf(trainDf.features))

In [23]:
_trainDf2.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- dvf: vector (nullable = true)



In [24]:
_trainDf2.show()

+-----+--------------------+------------------+
|label|            features|               dvf|
+-----+--------------------+------------------+
|  0.0| (4,[1,3],[1.0,5.5])| [0.0,1.0,0.0,5.5]|
|  1.0|(4,[0,2],[-1.0,0.5])|[-1.0,0.0,0.5,0.0]|
+-----+--------------------+------------------+



## 문제 S-1: RDD 훈련데이터 만들기

### 문제

머신러닝은 사람이 경험을 통해 배우는 것과 비슷하게 **과거 데이터로부터 학습**을 한다.
학습이란 어렵게 생각할 필요 없이, 과거 데이터에서 수학적이나 알고리즘을 활용하여 어떤 패턴을 찾아내는 것이다.
spark에서 제공한 **데이터 파일 ```data/mllib/sample_svm_data.txt```을 읽어서 훈련데이터**를 만들어 보자.

```python
1 0 2.52078447201548 0 0 0 2.004684436494304 2.000347299268466 0 2.228387042742021 2.228387042742023 0 0 0 0 0 0
0 2.857738033247042 0 0 2.619965104088255 0 2.004684436494304 2.000347299268466 0 2.228387042742021 2.228387042742023 0 0 0 0 0 0
0 2.857738033247042 0 2.061393766919624 0 0 2.004684436494304 0 0 2.228387042742021 2.228387042742023 0 0 0 0 0 0
1 0 0 2.061393766919624 2.619965104088255 0 2.004684436494304 2.000347299268466 0 0 0 0 2.055002875864414 0 0 0 0
1 2.857738033247042 0 2.061393766919624 2.619965104088255 0 2.004684436494304 0 0 0 0 0 2.055002875864414 0 0 0 0
```

### 해결

데이터를 읽어 **RDD**를 생성하고, ```label```, ```features```를 구성하여 ```Labeled Point```로 만든다.

### Python으로 파일 읽기

파일로부터 데이터를 읽기 위해, 파일명을 구성하고 ```try except``` 구문으로 입출력 오류를 확인할 수 있다.

In [4]:
import os

try:
    _fp=os.path.join(os.environ["SPARK_HOME"],\
        'data','mllib','sample_svm_data.txt')
except:
    print("An exception occurred")

파일로부터 데이터를 **```readlines()```** 함수로 모두 읽어 온다.
첫 행을 읽으면 label, features로 구성되어 있다.

In [5]:
_f=open(_fp,'r')
_lines=_f.readlines()
_f.close()

In [6]:
print _lines[0]

1 0 2.52078447201548 0 0 0 2.004684436494304 2.000347299268466 0 2.228387042742021 2.228387042742023 0 0 0 0 0 0



### Spark에서 RDD 생성

원본 데이터 ```sample_svm_data.txt```는 공백으로 구분되어 있다.
읽을 대상이 파일이므로, RDD를 사용한다. 각 행을 공백으로 분리하여 읽는다.

In [7]:
_rdd=spark.sparkContext.textFile(_fp)\
    .map(lambda line: [float(x) for x in line.split(' ')])

각 행으로 분리되므로 2차원 리스트가 생성이 된다. 첫째 행을 읽으려면 인덱스를 사용해야 한다.

In [11]:
_rdd.take(2)[0]

[1.0,
 0.0,
 2.52078447201548,
 0.0,
 0.0,
 0.0,
 2.004684436494304,
 2.000347299268466,
 0.0,
 2.228387042742021,
 2.228387042742023,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

### LabeledPoint 생성

위 데이터에서 보듯이 첫 열은 **label**로, 그 나머지는 **features**로 생성한다.

In [12]:
from pyspark.mllib.regression import LabeledPoint

_trainRdd0=_rdd.map(lambda line:LabeledPoint(line[0], line[1:]))

In [13]:
_trainRdd0.take(1)

[LabeledPoint(1.0, [0.0,2.52078447202,0.0,0.0,0.0,2.00468443649,2.00034729927,0.0,2.22838704274,2.22838704274,0.0,0.0,0.0,0.0,0.0,0.0])]

공백을 분리하고, 분리된 데이터를 labeled point로 구성하는 기능을 합쳐서 실행해 본다.

In [14]:
_trainRdd=spark.sparkContext.textFile(_fp)\
    .map(lambda line: [float(x) for x in line.split(' ')])\
    .map(lambda p:LabeledPoint(p[0], p[1:]))

In [15]:
_trainRdd.take(1)

[LabeledPoint(1.0, [0.0,2.52078447202,0.0,0.0,0.0,2.00468443649,2.00034729927,0.0,2.22838704274,2.22838704274,0.0,0.0,0.0,0.0,0.0,0.0])]

### 정리하면

데이터를 변환하는 과정을 함수로 만들었다.
```createLP(line)```는 행 데이터를 받아서 LabeledPoint로 생성하고 있다.

In [29]:
def createLP(line):
    p = [float(x) for x in line.split(' ')]
    return LabeledPoint(p[0], p[1:])

_rdd=spark.sparkContext.textFile(_fp)
trainRdd = _rdd.map(createLP)

trainRdd.take(1)

[LabeledPoint(1.0, [0.0,2.52078447202,0.0,0.0,0.0,2.00468443649,2.00034729927,0.0,2.22838704274,2.22838704274,0.0,0.0,0.0,0.0,0.0,0.0])]

### S.3.4 libsvm format

* svm을 처리하기 위한 데이터 형식이다.
* 0은 label, 나머지는 index:value 쌍으로 구성한다.
```python
[label] [index1]:[value1] [index2]:[value2] ...
[label] [index1]:[value1] [index2]:[value2] ...
```

* 예
```python
0 128:51 129:159 130:253 131:159 132:50 155:48 156:238 157:252 158:252 159:252 160:237 182:54 183:227 184:253 185:252 186:239 187:233 ...
```


In [3]:
fsvm=os.path.join(os.environ["SPARK_HOME"],'data','mllib','sample_libsvm_data.txt')
dfsvm = spark.read.format("libsvm").load(fsvm)

In [4]:
type(dfsvm)

pyspark.sql.dataframe.DataFrame

In [5]:
dfsvm.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [6]:
dfsvm.take(1)

[Row(label=0.0, features=SparseVector(692, {127: 51.0, 128: 159.0, 129: 253.0, 130: 159.0, 131: 50.0, 154: 48.0, 155: 238.0, 156: 252.0, 157: 252.0, 158: 252.0, 159: 237.0, 181: 54.0, 182: 227.0, 183: 253.0, 184: 252.0, 185: 239.0, 186: 233.0, 187: 252.0, 188: 57.0, 189: 6.0, 207: 10.0, 208: 60.0, 209: 224.0, 210: 252.0, 211: 253.0, 212: 252.0, 213: 202.0, 214: 84.0, 215: 252.0, 216: 253.0, 217: 122.0, 235: 163.0, 236: 252.0, 237: 252.0, 238: 252.0, 239: 253.0, 240: 252.0, 241: 252.0, 242: 96.0, 243: 189.0, 244: 253.0, 245: 167.0, 262: 51.0, 263: 238.0, 264: 253.0, 265: 253.0, 266: 190.0, 267: 114.0, 268: 253.0, 269: 228.0, 270: 47.0, 271: 79.0, 272: 255.0, 273: 168.0, 289: 48.0, 290: 238.0, 291: 252.0, 292: 252.0, 293: 179.0, 294: 12.0, 295: 75.0, 296: 121.0, 297: 21.0, 300: 253.0, 301: 243.0, 302: 50.0, 316: 38.0, 317: 165.0, 318: 253.0, 319: 233.0, 320: 208.0, 321: 84.0, 328: 253.0, 329: 252.0, 330: 165.0, 343: 7.0, 344: 178.0, 345: 252.0, 346: 240.0, 347: 71.0, 348: 19.0, 349: 28.0

## S.5 DataFrame 변환

DataFrame으로 만들어진 데이터를 변환해보자.
이러한 작업이 필요한 이유는 **기계학습에 넘겨줄 입력데이터를 형식에 맞추어야** 하기 때문이다.
데이터는 형식에 맞게 변환되고, 군집화, 회귀분석, 분류, 추천 모델 등에 입력으로 사용된다
물론 데이터는 '일련의 수' 또는 '텍스트'로 구성된다.
이런 데이터로부터 특징을 추출하여 **feature vectors**를 구성한다.
지도학습을 하는 경우에는 **class 또는 label** 값이 필요하다.

구분 | 컬럼 구성
-----|-----
DataFrame | 'label' (DoubleType), 'features' (sparse or dense vectors)
RDD | LabeledPoint

**RDD**를 만들고 나서도 데이터를 변환하기 위해 map-reduce와 같은 **transformation**, **action**을 사용하는 것과 같이,
**DataFrame**도 역시 **Transformer**, **Estimator**를 사용할 수 있다.
Estimator는 DataFrame에 적용되는 알고리즘을 말하는 것으로, Transformer를 생성해낸다.
Transformer는 DataFrame을 다른 DataFrame으로 변환하는 알고리즘을 말한다.

변환 기능 | 설명 | 함수
----------|----------|----------
Estimator | DataFrame에 적용해서 Transformer를 생성한다.| **```Estimator.fit()```**
Transformer | DataFrame을 적용해서 다른 DataFrame으로 생성한다. | **```Transformer.transform()```**

### S.5.1 텍스트 변환

#### Bag of Words 모델

텍스트를 단어의 집합, **'bag of words'**으로 구성된다고 보며, 단어의 **순서**는 의미를 가지지 않는다.

이런 영화리뷰가 있다고 하자.
"...그 영화는 매우 강렬했다. 그냥 좋았다. 영화관에서 보는 동안 긴장을 늦출 수 없었다. 갑돌이가 분장한 악당의 케릭터가 만들어지는 과정은 흥미롭지 않을 수가 없었다. 무비의 이야기 전개는 빠르고, 무엇이 진실이고 거짓인지 판단할 수 없었다. 누가 이런 영화를 좋아 하지 않을 수가 있겠는가 이모티콘..."

#### 단계

단계 1: 단어로 분할 Tokenization
그, 영화는, 매우, 강렬했다, 그냥, 좋았다, 영화관에서, 보는, 동안, 긴장을, 늦출, 수, 없었다, 갑돌이가, 분장한, 악당의, 케릭터가, 만들어지는, 과정은, 흥미롭지, 않을, 수가, 없었다, 무비의, 이야기, 전개는, 빠르고, 무엇이, 진실이고, 거짓인지, 판단할, 수, 없었다, 누가, 이런, 영화를, 좋아, 하지, 않을, 수가, 있겠는가, 이모티콘

단계 2: 정리
- 불필요, 오류 정리

단계 3: 불용어 stopwords 제거
- 그, 수, 수가, 수, 이런, 하지, 수가 등

단계 4: 어간 추출 stemming
영화는, 영화의는 다른 단어지만 조사를 제거하면 동일한 단어
- 좋았다, 좋아 단어들은 어근을 판별하면 동일한 단어이다.
- 영화, 무비의 단어는 이음동의

단계 5: 계량화
- word vector로 만든다.
- 있다-없다, 단어빈도, TF-IDF 사용할 수 있다.<br>dense, sparse 모두 가능하다.
```[1,1,1,1,1,0,0],[0,1,0,1,1,1,1]```

```python
[0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 1 1 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1]
[1 0 0 1 0 0 1 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 1 1 0 0 0 1 0 0 0 1 1 1 0 1 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1]
[0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 1]
```

### S.5.2 Python을 사용한 단어 빈도 계산

In [16]:
# Let it be lyrics
doc=[
    "When I find myself in times of trouble",
    "Mother Mary comes to me",
    "Speaking words of wisdom, let it be",
    "And in my hour of darkness",
    "She is standing right in front of me",
    "Speaking words of wisdom, let it be",
    "Let it be",
    "Let it be",
    "Let it be",
    "Let it be",
    "Whisper words of wisdom, let it be"
]

문서, 문장, 단어의 계층을 먼저 이해해야 한다.
문서는 문장으로 구성되어 있고, 문장은 단어로 구성되어 있다.
따라서 첫째 반복문은 문서의 각 문장에 대해, 단어로 분리하고 있다.
그 다음 반복문은 각 단어에 대해 빈도를 계산한다.
각 단어가 키가 되는데, **키가 존재하면 빈도를 증가하고, 존재하지 않으면 새로운 키를 생성**한다.

In [2]:
d={}
for sentence in doc:
    words=sentence.split()
    for word in words:
        if word in d:
            d[word]+=1
        else:
            d[word]=1

앞서 단어 빈도는 dictionary d에 저장하였다.
dictionary는 키, 빈도의 쌍으로 저장되어 있어서 ```iteritems()```으로 읽어낼 수 있다.

In [3]:
# Python 2 - 3 compatible code
# for k,v in d.items():
for k,v in d.iteritems():
    print k,v

right 1
be 7
is 1
When 1
it 7
in 3
Mary 1
Speaking 2
standing 1
darkness 1
find 1
wisdom, 3
to 1
Let 4
And 1
I 1
let 3
She 1
words 3
Mother 1
front 1
trouble 1
me 2
myself 1
hour 1
of 6
times 1
Whisper 1
my 1
comes 1


### S.5.3 Spark

텍스트를 2차원 배열로 만들어, DataFrame을 생성한다. schema는 만들어 주지 않아도 된다.

In [18]:
doc2d=[
    ["When I find myself in times of trouble"],
    ["Mother Mary comes to me"],
    ["Speaking words of wisdom, let it be"],
    ["And in my hour of darkness"],
    ["She is standing right in front of me"],
    ["Speaking words of wisdom, let it be"],
    [u"우리 Let it be"],
    [u"나 Let it be"],
    [u"너 Let it be"],
    ["Let it be"],
    ["Whisper words of wisdom, let it be"]
]

In [19]:
myDf=spark.createDataFrame(doc2d,['sent'])

```truncate=True```는 줄여서, ```False```는 출력을 줄이지 않고 출력한다.

In [20]:
myDf.show(truncate=True)

+--------------------+
|                sent|
+--------------------+
|When I find mysel...|
|Mother Mary comes...|
|Speaking words of...|
|And in my hour of...|
|She is standing r...|
|Speaking words of...|
|        우리 Let it be|
|         나 Let it be|
|         너 Let it be|
|           Let it be|
|Whisper words of ...|
+--------------------+



### S.5.4 Tokenizer

Tokenizer는 문장을 단어로 분리한다.
분리하는 기준은 whitespace로 공백, TAB, CR, New Line 등이 해당된다.
입력은 "sent"로 출력은 "words"로 한다.

구분 | 설명 | 예
----------|----------|----------|
corpus | 말뭉치 | "why she had to go", "where she have to go"
document | 레코드 | "why she had to go"
vocabularay | 중복없는 단어 집합 | "why","she","had","to","go","where","have"

In [21]:
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="sent", outputCol="words")

```transform()```은 앞서 만든 ```tokenizer```모델에 DataFrame을 변환하여 다른 DataFrame을 생성한다.
그 결과는 문자열 배열로 구성된다.

In [22]:
tokDf = tokenizer.transform(myDf)

In [23]:
tokDf.show(3)

+--------------------+--------------------+
|                sent|               words|
+--------------------+--------------------+
|When I find mysel...|[when, i, find, m...|
|Mother Mary comes...|[mother, mary, co...|
|Speaking words of...|[speaking, words,...|
+--------------------+--------------------+
only showing top 3 rows



```for```문으로 출력해보자. ```Row()``` 객체로 출력된다.

In [24]:
for r in tokDf.select("sent", "words").take(3):
    print r

Row(sent=u'When I find myself in times of trouble', words=[u'when', u'i', u'find', u'myself', u'in', u'times', u'of', u'trouble'])
Row(sent=u'Mother Mary comes to me', words=[u'mother', u'mary', u'comes', u'to', u'me'])
Row(sent=u'Speaking words of wisdom, let it be', words=[u'speaking', u'words', u'of', u'wisdom,', u'let', u'it', u'be'])


### S.5.5 RegTokenizer

Tokenizer는 white space로 분리하지만, RegexTokenizer는 단어를 분리하기 위해 **정규표현식**을 적용할 수 있다.
정규표현식을 사용하여 분리하거나 특정 패턴을 추출할 수 있다.
공백으로 분리할 경우 간단히 정규표현식 ```\s``` 패턴을 적용할 수 있다.
한글에는 ```\w``` 패턴이 적용되지 않는다.

In [25]:
from pyspark.ml.feature import RegexTokenizer
re = RegexTokenizer(inputCol="sent", outputCol="wordsReg", pattern="\\s+")

In [26]:
reDf=re.transform(myDf)
reDf.show()

+--------------------+--------------------+
|                sent|            wordsReg|
+--------------------+--------------------+
|When I find mysel...|[when, i, find, m...|
|Mother Mary comes...|[mother, mary, co...|
|Speaking words of...|[speaking, words,...|
|And in my hour of...|[and, in, my, hou...|
|She is standing r...|[she, is, standin...|
|Speaking words of...|[speaking, words,...|
|        우리 Let it be|   [우리, let, it, be]|
|         나 Let it be|    [나, let, it, be]|
|         너 Let it be|    [너, let, it, be]|
|           Let it be|       [let, it, be]|
|Whisper words of ...|[whisper, words, ...|
+--------------------+--------------------+



### S.5.6 Stopwords

텍스트를 분리하고 나면, 별 의미가 없거나 쓸모가 없는 단어들이 존재한다.
예를 들어 이, 그, 저와 같은 **한 단어** 또는 있다 등과 같은 **일부 동사**, 그래서, 그러나 등과 같은 **접속사** 등이 후보가 될 수 있다.
이런 불필요한 단어들을 불용어 Stopwords라고 하며, 입력데이터에서 제거하도록 한다.
영어의 경우 불용어가 식별되어 제공되고 있다
http://ir.dcs.gla.ac.uk/resources/linguistic_utils/stop_words


In [27]:
from pyspark.ml.feature import StopWordsRemover
stop = StopWordsRemover(inputCol="wordsReg", outputCol="nostops")

* 현재 stop words에 자신의 것을 추가해서, 재설정한다.

In [28]:
stopwords=list()
_stopwords=stop.getStopWords()
for e in _stopwords:
    stopwords.append(e)

_mystopwords=[u"나",u"너", u"우리"]
for e in _mystopwords:
    stopwords.append(e)
stop.setStopWords(stopwords)

StopWordsRemover_489b8b8ff0eb64c1cbf7

In [29]:
for e in stop.getStopWords():
    print e,

i me my myself we our ours ourselves you your yours yourself yourselves he him his himself she her hers herself it its itself they them their theirs themselves what which who whom this that these those am is are was were be been being have has had having do does did doing a an the and but if or because as until while of at by for with about against between into through during before after above below to from up down in out on off over under again further then once here there when where why how all any both each few more most other some such no nor not only own same so than too very s t can will just don should now i'll you'll he'll she'll we'll they'll i'd you'd he'd she'd we'd they'd i'm you're he's she's it's we're they're i've we've you've they've isn't aren't wasn't weren't haven't hasn't hadn't don't doesn't didn't won't wouldn't shan't shouldn't mustn't can't couldn't cannot could here's how's let's ought that's there's what's when's where's who's why's would 나 너 우리


* 한글의 stop words '너','우리'가 제거되었다.

In [30]:
stopDf=stop.transform(reDf)
stopDf.show()

+--------------------+--------------------+--------------------+
|                sent|            wordsReg|             nostops|
+--------------------+--------------------+--------------------+
|When I find mysel...|[when, i, find, m...|[find, times, tro...|
|Mother Mary comes...|[mother, mary, co...|[mother, mary, co...|
|Speaking words of...|[speaking, words,...|[speaking, words,...|
|And in my hour of...|[and, in, my, hou...|    [hour, darkness]|
|She is standing r...|[she, is, standin...|[standing, right,...|
|Speaking words of...|[speaking, words,...|[speaking, words,...|
|        우리 Let it be|   [우리, let, it, be]|               [let]|
|         나 Let it be|    [나, let, it, be]|               [let]|
|         너 Let it be|    [너, let, it, be]|               [let]|
|           Let it be|       [let, it, be]|               [let]|
|Whisper words of ...|[whisper, words, ...|[whisper, words, ...|
+--------------------+--------------------+--------------------+



### S.5.7 CountVectorizer

```CountVectorizer```는 텍스트를 입력해서, word vector를 출력한다.
우선 Tokenizer를 사용해서 단어로 분리하고난 후 사용한다.

* minDF
    * 소수점은 비율, 사용된 문서 수/전체 문서 수
        * 정수는 사용된 문서 수, 단어가 몇 개의 문서에 사용되어야 하는지

* 입력: a collection of text documents
* 출력: word vector (sparse) vocabulary x TF


3번째 문장 "Speaking words of wisdom, let it be"의 word vector를 구성해 본다.
id 값은 모든 문장에서 단어를 추출하고 나서야 부여된다.

단어 (3행 "Speaking words of wisdom, let it be") | id | 빈도 | 
-----|-----|-----
Speaking | 7 | 1
words | 13 | 1
of | stopword | 0
wisdom | 12 | 1
let | 3 | 1
it | stopword | 0
be | stopword | 0

문서와 단어별로 표를 만들 수 있다.
Let it be는 4번 반복이 되고 있다.
'be': 1 'it': 9 'let': 10를 확인해보자.

위 **word vector**를 표로 나타내면 아래와 같다.
행은 문장, 열은 id이다.
**3행은 doc2**이다. 해당하는 **단어 id의 빈도**를 적었다. 다른 행과 열은 이해를 돕기 위해 비워 놓았다.

```doc``` \ 단어 id  | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 |10 |11 |12 |13 |...
------|---|---|---|---|---|---|---|---|---|---|---|---|---|---
```doc 0``` |   |   |   |   |   |   |   |   |   |   |   |   |   |...
```doc 1``` |   |   |   |   |   |   |   |   |   |   |   |   |   |...
```doc 2``` |   |   | 1 |   |   |   | 1 |   |   |   |   | 1 | 1 |...
...   |   |   |   |   |   |   |   |   |   |   |   |   |   |...

#### sklearn CountVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words='english')

In [32]:
print vectorizer.fit_transform(doc)

  (0, 9)	1
  (0, 10)	1
  (1, 5)	1
  (1, 4)	1
  (1, 0)	1
  (2, 7)	1
  (2, 13)	1
  (2, 12)	1
  (2, 3)	1
  (3, 2)	1
  (3, 1)	1
  (4, 8)	1
  (4, 6)	1
  (5, 7)	1
  (5, 13)	1
  (5, 12)	1
  (5, 3)	1
  (6, 3)	1
  (7, 3)	1
  (8, 3)	1
  (9, 3)	1
  (10, 13)	1
  (10, 12)	1
  (10, 3)	1
  (10, 11)	1


In [33]:
print vectorizer.vocabulary_

{u'standing': 8, u'right': 6, u'darkness': 1, u'hour': 2, u'whisper': 11, u'times': 9, u'let': 3, u'speaking': 7, u'words': 13, u'mother': 5, u'trouble': 10, u'wisdom': 12, u'mary': 4, u'comes': 0}


In [34]:
print vectorizer.fit_transform(doc).todense()

[[0 0 0 0 0 0 0 0 0 1 1 0 0 0]
 [1 0 0 0 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 1 0 0 0 0 1 1]
 [0 1 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 1 0 0 0 0 1 1]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 1 1 1]]


#### Spark CountVectorizer

In [35]:
from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer(inputCol="nostops", outputCol="cv", vocabSize=30, minDF=1.0)

```CountVectorizerModel```은 ```fit()```하고 나면 얻어진다. 다음에 사용하는 ```HashingTF```는 ```fit()```하지 않는다는 점에서 차이가 있다.

In [36]:
cvModel = cv.fit(stopDf)

In [37]:
print type(cv),type(cvModel)

<class 'pyspark.ml.feature.CountVectorizer'> <class 'pyspark.ml.feature.CountVectorizerModel'>


In [38]:
cvDf = cvModel.transform(stopDf)

In [39]:
cvDf.show(3)

+--------------------+--------------------+--------------------+--------------------+
|                sent|            wordsReg|             nostops|                  cv|
+--------------------+--------------------+--------------------+--------------------+
|When I find mysel...|[when, i, find, m...|[find, times, tro...|(16,[5,6,8],[1.0,...|
|Mother Mary comes...|[mother, mary, co...|[mother, mary, co...|(16,[10,13,14],[1...|
|Speaking words of...|[speaking, words,...|[speaking, words,...|(16,[0,1,2,3],[1....|
+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



이 가운데 일부 컬럼만을 선택하여 출력할 수 있다.
```(16,[5,6,8],[1.0,1.0,1.0])```
16은 전체 단어의 개수, 그리고 다음 5,6,8은 값이 있는 컬럼 번호, 1.0,1.0,1.0은 그 값을 말한다.

In [40]:
cvDf.select('sent','nostops','cv').show()

+--------------------+--------------------+--------------------+
|                sent|             nostops|                  cv|
+--------------------+--------------------+--------------------+
|When I find mysel...|[find, times, tro...|(16,[5,6,8],[1.0,...|
|Mother Mary comes...|[mother, mary, co...|(16,[10,13,14],[1...|
|Speaking words of...|[speaking, words,...|(16,[0,1,2,3],[1....|
|And in my hour of...|    [hour, darkness]|(16,[7,9],[1.0,1.0])|
|She is standing r...|[standing, right,...|(16,[4,12,15],[1....|
|Speaking words of...|[speaking, words,...|(16,[0,1,2,3],[1....|
|        우리 Let it be|               [let]|      (16,[0],[1.0])|
|         나 Let it be|               [let]|      (16,[0],[1.0])|
|         너 Let it be|               [let]|      (16,[0],[1.0])|
|           Let it be|               [let]|      (16,[0],[1.0])|
|Whisper words of ...|[whisper, words, ...|(16,[0,1,2,11],[1...|
+--------------------+--------------------+--------------------+



```CountVectorizer```에서 사용된 단어 목록을 출력할 수 있다. 아래 단어의 수를 세어보면 위 sparse vector의 컬럼 개수와 동일하다.

In [41]:
cvModel.vocabulary

[u'let',
 u'wisdom,',
 u'words',
 u'speaking',
 u'right',
 u'trouble',
 u'find',
 u'hour',
 u'times',
 u'darkness',
 u'mother',
 u'whisper',
 u'front',
 u'mary',
 u'comes',
 u'standing']

### S.5.8 TF-IDF

```TfidfTransformer```는 **TF-IDF(Term Frequency-Inverse Document Frequency)**를 계산한다.
이를 위해서는 우선 Tokenizer를 사용하여 문장을 단어로 분리해 놓아야 한다.

HashingTF를 사용하여 'word vector'를 계산한다.
HashingTF은 hash함수에 따라 단어의 고유번호를 생성하며, hash고유번호의 충돌 가능성을 줄이기 위해, 단어 수를 제한할 수 있다.
그리고 IDF를 계산하고, TF-IDF를 계산한다.

#### S.5.8.1 TF-IDF 계산

'Let it be'가사 세 번째 줄 **'wisdom' 단어**의 TF-IDF를 계산해보자.
**TF**는 단어빈도수, 즉 문서에 단어가 나타난 빈도수를 의미한다.
단어빈도는 경우에 따라서는 문제가 될 수 있다. 예를 들어, 'a', 'the', 'of'와 같은 단어는 빈도는 높지만 별로 유용하지 못하다.
이 경우 IDF는 유용하다. **IDF**는 자주 나타나는 단어에 대한 가중치를 줄이고, 드물게 나타나는 단어에 가중치를 높이는 방식으로 계산된다.

t는 단어, 문서는 d, D는 corpus,

항목 | 설명 | 예제
-----|-----|-----
tf(d,f) | 단어 t가 문서 d에서 나타나는 단어의 빈도 수, term frequency | $f_{t,d}$ / (number of words in d) = 1/4 = 0.25<br>(3번째 문서에 stopwords를 제외하면 4개의 단어, wisdom은 1회 나타난다.)
df | document frequency 단어가 나타난 문서 수 | 3 (wisdom이 포함된 문서는 3)
N | number of documents 전체 문서의 수 | 11 (전체의 문서는 11개)
idf | inverse document frequency 단어가 나타난 문서의 비율을 거꾸로 | ln(N+1 / df+1) + 1 = log(12/4) + 1 = 1.09861 + 1<br>0으로 나뉘는 것을 방지하기 위해 **smoothing**, 즉 1을 더한다. 

프로그래밍에서는 메모리를 적게 사용하도록 설계되어 있다. ```1/4```와 같이 정수 타잎으로 연산하면, 정수를 사용하여 연산하고 그 결과도 정수를 출력하게 된다. 이를 변환하기 위해 ```1.```의 경우에서와 같이 소수를 사용하자.

In [11]:
import math
tf=1./4
df=3.
N=11.
idf=math.log((N+1)/(df+1))+1
print idf

2.09861228867


#### S.5.8.2 sklearn을 사용한 TF-IDF

우선 'sklearn'의 TF-IDF를 계산해보자.
**```CountVectorizer```**는 텍스트를 단어의 빈도로 변환해주어, 문서 x 단어 표를 출력할 수 있다.
**```CountVectorizer()```**의 인자로
analyzer ("word", "character ngram" 등 선택),
tokenizer (단어의 tokenizer를 지정),
stop_words (불용어 처리 기준),
max_features (최대 속성 개수) 등을 지정할 수 있다.
그 다음으로, TF-IDF를 계산할 수 있다. 이 때 (문서id, 단어id) 별로 결과가 출력된다.

```TfidfVectorizer```를 사용해서 계산하면 그 결과를 아래와 같이 볼 수 있다.
```python
(2,12) 2.09861228867
```

결과에서
**'2'**는 3번째 문서번호, **'12'**는 'wisdom' 단어번호
TF-IDF는 ```2.09861228867```이다.

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_df=1.0, min_df=1, stop_words='english',norm = None)

In [43]:
print vectorizer.fit_transform(doc)

  (0, 10)	2.79175946923
  (0, 9)	2.79175946923
  (1, 0)	2.79175946923
  (1, 4)	2.79175946923
  (1, 5)	2.79175946923
  (2, 3)	1.40546510811
  (2, 12)	2.09861228867
  (2, 13)	2.09861228867
  (2, 7)	2.38629436112
  (3, 1)	2.79175946923
  (3, 2)	2.79175946923
  (4, 6)	2.79175946923
  (4, 8)	2.79175946923
  (5, 3)	1.40546510811
  (5, 12)	2.09861228867
  (5, 13)	2.09861228867
  (5, 7)	2.38629436112
  (6, 3)	1.40546510811
  (7, 3)	1.40546510811
  (8, 3)	1.40546510811
  (9, 3)	1.40546510811
  (10, 11)	2.79175946923
  (10, 3)	1.40546510811
  (10, 12)	2.09861228867
  (10, 13)	2.09861228867


In [44]:
print vectorizer.vocabulary_

{u'standing': 8, u'right': 6, u'darkness': 1, u'hour': 2, u'whisper': 11, u'times': 9, u'let': 3, u'speaking': 7, u'words': 13, u'mother': 5, u'trouble': 10, u'wisdom': 12, u'mary': 4, u'comes': 0}


In [45]:
print vectorizer.idf_

[ 2.79175947  2.79175947  2.79175947  1.40546511  2.79175947  2.79175947
  2.79175947  2.38629436  2.79175947  2.79175947  2.79175947  2.79175947
  2.09861229  2.09861229]


#### S.5.8.3 Spark를 사용한 TF-IDF

HashingTF는 단어집합을 워드벡터 word vector로 변환하는데, 해시함수를 사용해서 단어에 해당하는 일련번호를 결정한다.
HashingTF에서의 **```numFeatures```는 $2^n$**으로 결정하게 된다.
단어 갯수가 900이면, $2^{10}=1024$이므로 1024로 설정하면 된다.
기본은 $2^{18}=262,144$이다.

In [39]:
from pyspark.ml.feature import HashingTF, IDF

hashTF = HashingTF(inputCol="nostops", outputCol="hash", numFeatures=64)

```HashingTF```는 ```fit()```하지 않고 ```transform()``` 한다.

In [40]:
hashDf = hashTF.transform(stopDf)

```(50,[10,24,43],[1.0,1.0,1.0])```
16은 해시 개수 (앞서 CountVectorizer의 경우에서와 같이 전체 단어의 개수가 아니다), 그리고 다음 [10,24,43]은 값이 있는 **해시 컬럼** 번호, 1.0,1.0,1.0은 그 값을 말한다.

In [41]:
hashDf.show()

+--------------------+--------------------+--------------------+--------------------+
|                sent|            wordsReg|             nostops|                hash|
+--------------------+--------------------+--------------------+--------------------+
|When I find mysel...|[when, i, find, m...|[find, times, tro...|(50,[10,24,43],[1...|
|Mother Mary comes...|[mother, mary, co...|[mother, mary, co...|(50,[1,21,24],[1....|
|Speaking words of...|[speaking, words,...|[speaking, words,...|(50,[9,12,14,41],...|
|And in my hour of...|[and, in, my, hou...|    [hour, darkness]|(50,[23,27],[1.0,...|
|She is standing r...|[she, is, standin...|[standing, right,...|(50,[24,43,46],[1...|
|Speaking words of...|[speaking, words,...|[speaking, words,...|(50,[9,12,14,41],...|
|        우리 Let it be|   [우리, let, it, be]|               [let]|     (50,[14],[1.0])|
|         나 Let it be|    [나, let, it, be]|               [let]|     (50,[14],[1.0])|
|         너 Let it be|    [너, let, it, be]|           

tf-idf

In [ ]:
idf = IDF(inputCol="hash", outputCol="idf")

In [ ]:
idfModel = idf.fit(hashDf)
idfDf = idfModel.transform(hashDf)

In [30]:
for e in idfDf.select("nostops","hash").take(10):
    print(e)

Row(nostops=[u'find', u'times', u'trouble'], hash=SparseVector(50, {10: 1.0, 24: 1.0, 43: 1.0}))
Row(nostops=[u'mother', u'mary', u'comes'], hash=SparseVector(50, {1: 1.0, 21: 1.0, 24: 1.0}))
Row(nostops=[u'speaking', u'words', u'wisdom,', u'let'], hash=SparseVector(50, {9: 1.0, 12: 1.0, 14: 1.0, 41: 1.0}))
Row(nostops=[u'hour', u'darkness'], hash=SparseVector(50, {23: 1.0, 27: 1.0}))
Row(nostops=[u'standing', u'right', u'front'], hash=SparseVector(50, {24: 1.0, 43: 1.0, 46: 1.0}))
Row(nostops=[u'speaking', u'words', u'wisdom,', u'let'], hash=SparseVector(50, {9: 1.0, 12: 1.0, 14: 1.0, 41: 1.0}))
Row(nostops=[u'let'], hash=SparseVector(50, {14: 1.0}))
Row(nostops=[u'let'], hash=SparseVector(50, {14: 1.0}))
Row(nostops=[u'let'], hash=SparseVector(50, {14: 1.0}))
Row(nostops=[u'let'], hash=SparseVector(50, {14: 1.0}))


#### S.5.10 Word2Vec

* see wikipedia https://en.wikipedia.org/wiki/Word2vec

In [207]:
from pyspark.ml.feature import Word2Vec
word2Vec = Word2Vec(vectorSize=3,minCount=0,inputCol="words",outputCol="w2v")
model = word2Vec.fit(tokDf)
w2vDf = model.transform(tokDf)
for e in w2vDf.select("w2v").take(3):
    print(e)

Row(w2v=DenseVector([-0.0367, 0.0097, 0.0479]))
Row(w2v=DenseVector([-0.0482, 0.0223, 0.0095]))
Row(w2v=DenseVector([0.052, -0.001, -0.0019]))


#### S.5.11 NGram

* unigram은 한 단어로, bigram은 두 단어로 구성한다.

In [41]:
from pyspark.ml.feature import NGram
ngram = NGram(n=2, inputCol="words", outputCol="ngrams")
ngramDf = ngram.transform(tokDf)
ngramDf.show()
for e in ngramDf.select("words","ngrams").take(3):
    print e

+--------------------+--------------------+--------------------+
|                sent|               words|              ngrams|
+--------------------+--------------------+--------------------+
|When I find mysel...|[when, i, find, m...|[when i, i find, ...|
|Mother Mary comes...|[mother, mary, co...|[mother mary, mar...|
|Speaking words of...|[speaking, words,...|[speaking words, ...|
|And in my hour of...|[and, in, my, hou...|[and in, in my, m...|
|She is standing r...|[she, is, standin...|[she is, is stand...|
|Speaking words of...|[speaking, words,...|[speaking words, ...|
|        우리 Let it be|   [우리, let, it, be]|[우리 let, let it, ...|
|         나 Let it be|    [나, let, it, be]|[나 let, let it, i...|
|         너 Let it be|    [너, let, it, be]|[너 let, let it, i...|
|           Let it be|       [let, it, be]|     [let it, it be]|
|Whisper words of ...|[whisper, words, ...|[whisper words, w...|
+--------------------+--------------------+--------------------+

Row(words=[u'when', u'i'

### S.5.4 StringIndexer

문자열 컬럼을 인덱스 컬럼으로 변환한다. **빈도가 제일 높은 순서**로 ```0.0```부터 인덱스 값이 주어진다. 인덱스는 double 형을 가지게 된다.
없는 레이블에 대해서는 예외가 발생할 수 있으므로 (default), ```setHandleInvalid("skip")``` 함수로 'skip', 'keep', 'error' 등 처리

구분 | 설명 | 예
-----|-----|-----
nominal | 명목 또는 구분 값 cateogry  | 사자, 호랑이, 사람
ordinal | 명목값과 다른 점은 순서가 있다. | 키 low, med, high
interval | 일정한 간격이 있다. | 150-165, 165-180, 180-195


from pyspark.ml.feature import StringIndexer
labelIndexer = StringIndexer(inputCol="sent", outputCol="sentLabel")
model=labelIndexer.fit(myDf)
siDf=model.transform(myDf)
siDf.show()

#### S.5.12 연속데이터의 변환

몸무게(inches), 키(pounds) 데이터를 분석해보자.
이 데이터는 정량, 연속 데이터이다. 
출처는 https://people.sc.fsu.edu/~jburkardt/data/csv/hw_200.csv

```python
1	65.78	112.99
2	71.52	136.49
3	69.40	153.03
4	68.22	142.34
5	67.79	144.30
6	68.70	123.30
7	69.80	141.49
8	70.01	136.46
9	67.90	112.37
10	66.78	120.67
11	66.49	127.45
12	67.62	114.14
13	68.30	125.61
14	67.12	122.46
15	68.28	116.09
16	71.09	140.00
17	66.46	129.50
18	68.65	142.97
19	71.23	137.90
20	67.13	124.04
21	67.83	141.28
22	68.88	143.54
23	63.48	97.90
24	68.42	129.50
25	67.63	141.85
26	67.21	129.72
27	70.84	142.42
28	67.49	131.55
29	66.53	108.33
30	65.44	113.89
31	69.52	103.30
32	65.81	120.75
33	67.82	125.79
34	70.60	136.22
35	71.80	140.10
36	69.21	128.75
37	66.80	141.80
38	67.66	121.23
39	67.81	131.35
40	64.05	106.71
41	68.57	124.36
42	65.18	124.86
43	69.66	139.67
44	67.97	137.37
45	65.98	106.45
46	68.67	128.76
47	66.88	145.68
48	67.70	116.82
49	69.82	143.62
50	69.09	134.93
```

In [67]:
from pyspark.sql.types import *
rdd=spark.sparkContext\
    .textFile(os.path.join('data','ds_spark_heightweight.txt'))

myRdd=rdd.map(lambda line:[float(x) for x in line.split('\t')])
myDf=spark.createDataFrame(myRdd,["id","weight","height"])

In [54]:
myDf.printSchema()

root
 |-- id: double (nullable = true)
 |-- weight: double (nullable = true)
 |-- height: double (nullable = true)



In [68]:
from pyspark.ml.feature import Binarizer
binarizer = Binarizer(threshold=68.0, inputCol="weight", outputCol="weight2")
binDf = binarizer.transform(myDf)
binDf.show(10)

+----+------+------+-------+
|  id|weight|height|weight2|
+----+------+------+-------+
| 1.0| 65.78|112.99|    0.0|
| 2.0| 71.52|136.49|    1.0|
| 3.0|  69.4|153.03|    1.0|
| 4.0| 68.22|142.34|    1.0|
| 5.0| 67.79| 144.3|    0.0|
| 6.0|  68.7| 123.3|    1.0|
| 7.0|  69.8|141.49|    1.0|
| 8.0| 70.01|136.46|    1.0|
| 9.0|  67.9|112.37|    0.0|
|10.0| 66.78|120.67|    0.0|
+----+------+------+-------+
only showing top 10 rows



In [69]:
from pyspark.ml.feature import QuantileDiscretizer

discretizer = QuantileDiscretizer(numBuckets=3, inputCol="height", outputCol="height3")
qdDf = discretizer.fit(binDf).transform(binDf)
qdDf.show(10)


+----+------+------+-------+-------+
|  id|weight|height|weight2|height3|
+----+------+------+-------+-------+
| 1.0| 65.78|112.99|    0.0|    0.0|
| 2.0| 71.52|136.49|    1.0|    1.0|
| 3.0|  69.4|153.03|    1.0|    2.0|
| 4.0| 68.22|142.34|    1.0|    2.0|
| 5.0| 67.79| 144.3|    0.0|    2.0|
| 6.0|  68.7| 123.3|    1.0|    0.0|
| 7.0|  69.8|141.49|    1.0|    2.0|
| 8.0| 70.01|136.46|    1.0|    1.0|
| 9.0|  67.9|112.37|    0.0|    0.0|
|10.0| 66.78|120.67|    0.0|    0.0|
+----+------+------+-------+-------+
only showing top 10 rows



#### 5.5.13 VectorAssembler

* 열을 묶어서 Vector열로 만든다.
* string은 묶을 수 없다.
* pyspark.ml.linalg.Vectors를 사용한다. (주의: pyspark.mllib.linalg.Vectors를 사용하지 않는다.)

In [70]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

va = VectorAssembler(inputCols=["weight2","height3"],outputCol="features")
vaDf = va.transform(qdDf)
vaDf.printSchema()
vaDf.show(5)

root
 |-- id: double (nullable = true)
 |-- weight: double (nullable = true)
 |-- height: double (nullable = true)
 |-- weight2: double (nullable = true)
 |-- height3: double (nullable = true)
 |-- features: vector (nullable = true)

+---+------+------+-------+-------+---------+
| id|weight|height|weight2|height3| features|
+---+------+------+-------+-------+---------+
|1.0| 65.78|112.99|    0.0|    0.0|(2,[],[])|
|2.0| 71.52|136.49|    1.0|    1.0|[1.0,1.0]|
|3.0|  69.4|153.03|    1.0|    2.0|[1.0,2.0]|
|4.0| 68.22|142.34|    1.0|    2.0|[1.0,2.0]|
|5.0| 67.79| 144.3|    0.0|    2.0|[0.0,2.0]|
+---+------+------+-------+-------+---------+
only showing top 5 rows



#### S.5.14 Pipeline

**Pipeline**은 여러 Estimator를 묶은 Estimator를 반환한다. 단계적으로 Estimator를 적용하기 위해 사용한다.

* Pipeline은 여러 작업을 묶어, 순서대로 단계적으로 처리한다.

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.classification import LogisticRegression

df = spark.createDataFrame([
    (0L, "a b c d e spark", 1.0),
    (1L, "b d", 0.0),
    (2L, "spark f g h", 1.0),
    (3L, "hadoop mapreduce", 0.0),
    (4L, "my dog has flea problems. help please.",0.0)
    ], ["id", "text", "label"])

tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.01)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

# Fit the pipeline to training documents.
model = pipeline.fit(df)
myDf = model.transform(df)

## 문제 S-4: 연설문을 기계학습하기 위해 변환

한 글자 단어를 제외하고, 단어의 TF-IDF를 계산해서 features로 구성한다.

In [17]:
import os
#spark.createDataFrame(os.path.join("data", "20191021_policeAddress.txt"))
#df=spark.read.text(os.path.join("data", "20191021_policeAddress.txt"))

from pyspark.sql.types import StructType, StructField, StringType
police=spark.read\
    .options(header="true", delimiter=" ", inferSchema="true")\
    .schema(
        StructType([
            StructField("sent",StringType()),
            ])
    )\
    .text(os.path.join("data", "20191021_policeAddress.txt")) 

문장이 잘리지 않고 온전하게 출력되려면 'False'로 설정한다.

In [18]:
police.show(5, False)

+-------------------------------------------------------------------------------------------------------------------------------------+
|sent                                                                                                                                 |
+-------------------------------------------------------------------------------------------------------------------------------------+
|존경하는 국민 여러분, 경찰관 여러분, 일흔네 돌 ‘경찰의 날’입니다.                                                                                              |
|                                                                                                                                     |
|국민의 안전을 위해 밤낮없이 애쓰시는 전국의 15만 경찰관 여러분께 먼저 감사를 드립니다. 전몰·순직 경찰관들의 고귀한 희생에 경의를 표합니다. 유가족 여러분께 위로의 마음을 전합니다.                              |
|                                                                                                                                     |
|오늘 홍조근정훈장을 받으신 중앙경찰학교장 이은정 치안감님, 근정포장을 받으신 광주남부

In [9]:
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="sent", outputCol="words")
tokDf = tokenizer.transform(police)

In [10]:
tokDf.show(5, False)

+-------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|sent                                                                                                                                 |words                                                                                                                                                            |
+-------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|존경하는 국민 여러분, 경찰관 여러분, 일흔네 돌 ‘경찰의 날’입니다.                                                                  

In [24]:
for r in tokDf.select("sent").take(3):
    print r[0]

존경하는 국민 여러분, 경찰관 여러분, 일흔네 돌 ‘경찰의 날’입니다.
 
국민의 안전을 위해 밤낮없이 애쓰시는 전국의 15만 경찰관 여러분께 먼저 감사를 드립니다. 전몰·순직 경찰관들의 고귀한 희생에 경의를 표합니다. 유가족 여러분께 위로의 마음을 전합니다.


In [46]:
from pyspark.ml.feature import StopWordsRemover
stop = StopWordsRemover(inputCol="words", outputCol="nostops")

In [47]:
stop.setStopWords([u"돌",u"너", u"우리"])

StopWordsRemover_4e989379a80b9a486107

In [48]:
_mystopwords=[u"나",u"너", u"우리"]
for e in _mystopwords:
    stopwords.append(e)
stop.setStopWords(stopwords)

StopWordsRemover_4e989379a80b9a486107

In [33]:
stopDf=stop.transform(tokDf)
stopDf.show()

+--------------------+--------------------+--------------------+
|                sent|               words|             nostops|
+--------------------+--------------------+--------------------+
|존경하는 국민 여러분, 경찰관 ...|[존경하는, 국민, 여러분,, ...|[존경하는, 국민, 여러분,, ...|
|                    |                  []|                  []|
|국민의 안전을 위해 밤낮없이 애...|[국민의, 안전을, 위해, 밤낮...|[국민의, 안전을, 위해, 밤낮...|
|                    |                  []|                  []|
|오늘 홍조근정훈장을 받으신 중앙...|[오늘, 홍조근정훈장을, 받으신...|[오늘, 홍조근정훈장을, 받으신...|
|                    |                  []|                  []|
|       사랑하는 경찰관 여러분,|   [사랑하는, 경찰관, 여러분,]|   [사랑하는, 경찰관, 여러분,]|
|                    |                  []|                  []|
|여러분의 헌신적 노력으로 우리의...|[여러분의, 헌신적, 노력으로,...|[여러분의, 헌신적, 노력으로,...|
|                    |                  []|                  []|
|치안의 개선은 국민의 체감으로 ...|[치안의, 개선은, 국민의, 체...|[치안의, 개선은, 국민의, 체...|
|                    |                  []|                  []|
|한국을 찾는 외국 관광객들도 우...|[한국

In [49]:
for r in stopDf.select("nostops").take(3):
    for e in r:
        print e

[u'find', u'times', u'trouble']
[u'mother', u'mary', u'comes']
[u'speaking', u'words', u'wisdom,', u'let']


## 문제 텍스트

http://help.sentiment140.com/for-students/

* 트윗 정서 (0=부정, 2=중립, 4=긍정)
* 트윗 ID
* 트윗 일자
* 조회 (없으면 NO_QUERY)
* 사용자
* 트윗 텍스트

1) 전체 데이터 갯수, 각 '부정' '긍정' '중립' 별 데이터 갯수
2) 트윗 정서 컬럼을 'label'로 변경
5) stopwords 제거하고, 출력
6) pipeline으로 tf-idf 계산하고 'features' 컬럼으로 변경

https://github.com/tthustla/twitter_sentiment_analysis_part2/blob/master/Capstone_part3-Copy1.ipynb
https://towardsdatascience.com/another-twitter-sentiment-analysis-with-python-part-2-333514854913

In [3]:
import os
os.getcwd()

'/home/jsl/Code/git/bb/jsl/pyds'

In [4]:
tweet = spark.read\
    .format('com.databricks.spark.csv')\
    .options(header='false', inferschema='true')\
    .load(os.path.join("data","stanford", "testdata.manual.2009.06.14.csv"))

In [4]:
tweet.columns

['_c0', '_c1', '_c2', '_c3', '_c4', '_c5']

In [5]:
tweet.count()

498

### 결측값

In [7]:
def countNull(df,var):
    return df.where(df[var].isNull()).count()

missing = {c: countNull(tweet,c) for c in ['_c0', '_c1', '_c2', '_c3', '_c4', '_c5']}

print missing

{'_c1': 0, '_c0': 0, '_c3': 0, '_c2': 0, '_c5': 0, '_c4': 0}


In [5]:
tweet.show()

+---+---+--------------------+-------+--------------+--------------------+
|_c0|_c1|                 _c2|    _c3|           _c4|                 _c5|
+---+---+--------------------+-------+--------------+--------------------+
|  4|  3|Mon May 11 03:17:...|kindle2|        tpryan|@stellargirl I lo...|
|  4|  4|Mon May 11 03:18:...|kindle2|        vcu451|Reading my kindle...|
|  4|  5|Mon May 11 03:18:...|kindle2|        chadfu|Ok, first assesme...|
|  4|  6|Mon May 11 03:19:...|kindle2|         SIX15|@kenburbary You'l...|
|  4|  7|Mon May 11 03:21:...|kindle2|      yamarama|@mikefish  Fair e...|
|  4|  8|Mon May 11 03:22:...|kindle2|  GeorgeVHulme|@richardebaker no...|
|  0|  9|Mon May 11 03:22:...|    aig|       Seth937|Fuck this economy...|
|  4| 10|Mon May 11 03:26:...| jquery|     dcostalis|Jquery is my new ...|
|  4| 11|Mon May 11 03:27:...|twitter|       PJ_King|       Loves twitter|
|  4| 12|Mon May 11 03:29:...|  obama|   mandanicole|how can you not l...|
|  2| 13|Mon May 11 03:32

In [7]:
tweet.select('_c5').show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------+
|_c5                                                                                                                                         |
+--------------------------------------------------------------------------------------------------------------------------------------------+
|@stellargirl I loooooooovvvvvveee my Kindle2. Not that the DX is cool, but the 2 is fantastic in its own right.                             |
|Reading my kindle2...  Love it... Lee childs is good read.                                                                                  |
|Ok, first assesment of the #kindle2 ...it fucking rocks!!!                                                                                  |
|@kenburbary You'll love your Kindle2. I've had mine for a few months and never looked back. The new big one is huge! No need for remorse! :)|

### 정규표현식


* I've -> I have
* loooooveee -> love
* 단어 뒤에 !!! 이런 거 제외

In [10]:
import re
from pyspark.sql.types import *
from pyspark.sql.functions import udf

In [11]:
def get_date_day(a):
    x, y = re.split('^([\d]+-[\d]+-[\d]+)', a)[1:]
    return [x, y[1:]]

In [12]:
get_date_day('20-13-2012-monday')

['20-13-2012', 'monday']

In [ ]:
get_date_udf = udf(get_date_day, ArrayType(StringType()))

In [ ]:
>>> df = sc.parallelize([('20-13-2012-monday',), ('20-14-2012-tues',), ('20-13-2012-wed',)]).toDF(['A'])
>>> df.show()
+-----------------+
|                A|
+-----------------+
|20-13-2012-monday|
|  20-14-2012-tues|
|   20-13-2012-wed|
+-----------------+

>>> df = df.withColumn("A12", get_date_udf('A'))
>>> df.show(truncate=False)
+-----------------+--------------------+
|A                |A12                 |
+-----------------+--------------------+
|20-13-2012-monday|[20-13-2012, monday]|
|20-14-2012-tues  |[20-14-2012, tues]  |
|20-13-2012-wed   |[20-13-2012, wed]   |
+-----------------+--------------------+

>>> df = df.withColumn("A1", udf(lambda x:x[0])('A12')).withColumn("A2", udf(lambda x:x[1])('A12'))
>>> df = df.drop('A12')
>>> df.show(truncate=False)
+-----------------+----------+------+
|A                |A1        |A2    |
+-----------------+----------+------+
|20-13-2012-monday|20-13-2012|monday|
|20-14-2012-tues  |20-14-2012|tues  |
|20-13-2012-wed   |20-13-2012|wed   |
+-----------------+----------+------+

### Rename

In [9]:
tweet=tweet.withColumnRenamed("_c0", "label")

tweet.filter((tweet.label != 2) & (df.bar != 'b'))

In [101]:
tweet=tweet.filter(tweet.label != 2)

In [105]:
tweet.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|    4|  182|
|    0|  177|
+-----+-----+



In [82]:
(train, val, test) = tweet.randomSplit([0.6, 0.2, 0.2], seed = 2000)

In [83]:
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="_c5", outputCol="words")
#tokDf = tokenizer.transform(train)

In [84]:
from pyspark.ml.feature import StopWordsRemover
stopwords=list()
stop = StopWordsRemover(inputCol="words", outputCol="nostops")
stop.setStopWords([u"돌",u"너", u"우리"])
_mystopwords=[u"나",u"너", u"우리"]
for e in _mystopwords:
    stopwords.append(e)
stop.setStopWords(stopwords)
#stopDf=stop.transform(tokDf)
#stopDf.show()

StopWordsRemover_401e9ef90eea58d7de36

In [85]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

hashtf = HashingTF(numFeatures=2**16, inputCol="words", outputCol='tf')
idf = IDF(inputCol='tf', outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
#label_stringIdx = StringIndexer(inputCol = "target", outputCol = "label")
pipeline = Pipeline(stages=[tokenizer, stop, hashtf, idf])

pipelineFit = pipeline.fit(train)

In [86]:
trainDf = pipelineFit.transform(train)
valDf = pipelineFit.transform(val)
trainDf.show(5)

+-----+---+--------------------+------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|label|_c1|                 _c2|   _c3|           _c4|                 _c5|               words|             nostops|                  tf|            features|
+-----+---+--------------------+------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    0|  9|Mon May 11 03:22:...|   aig|       Seth937|Fuck this economy...|[fuck, this, econ...|[fuck, this, econ...|(65536,[7173,1607...|(65536,[7173,1607...|
|    0| 14|Mon May 11 03:32:...| obama|   kylesellers|@Karoli I firmly ...|[@karoli, i, firm...|[@karoli, i, firm...|(65536,[8436,9287...|(65536,[8436,9287...|
|    0| 17|Mon May 11 05:06:...|  nike|   vincentx24x|dear nike, stop w...|[dear, nike,, sto...|[dear, nike,, sto...|(65536,[2830,5660...|(65536,[2830,5660...|
|    0| 19|Mon May 11 05:20:...|lebron| 

In [87]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(maxIter=100)
lrModel = lr.fit(trainDf)
predictions = lrModel.transform(valDf)

In [89]:
predictions.select('label','prediction').show(1000)

+-----+----------+
|label|prediction|
+-----+----------+
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    4|       4.0|
|    4|       4.0|
|    4|       4.0|
|    4|       4.0|
|    4|       4.0|
|    4|       4.0|
|    4|       4.0|
|    4|       4.0|
|    4|       4.0|
|    4|       4.0|
|    4|     

In [90]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(predictions)

0.5

In [91]:
accuracy = predictions.filter(predictions.label == predictions.prediction).count() / float(val.count())
accuracy

0.4507042253521127